In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt 
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity
from torch.utils.data import TensorDataset, DataLoader

## 5. 核心模式 (Core Pattern)

### 5.1 混合驱动的优化 (Hybrid-Driven Optimization)

| 驱动力 | 角色 | 作用方式 |
|---|---|---|
| **Data-Driven** | 已知的稀疏观测点 | 通过 `loss_data = loss_fn(u_pred, u_label)` 保证模型**拟合观测数据** |
| **Physics-Informed** | 无处不在的 PDE | 通过 `loss_pde = loss_fn(residual, 0)` 强制模型**遵守物理定律** |

> 两股力量共同作用：  
> `total_loss = loss_data + λ * loss_pde`  
> 引导参数走向**既拟合数据又满足方程**的最优解。

---

### 5.2 物理定律 → 可优化损失函数

| 步骤 | 技术 | 说明 |
|---|---|---|
| **方程** | $u_t + u u_x - \nu u_{xx} = 0$ | Burgers 方程 |
| **残差** | `residual = compute_pde_residual(x, t)` | 利用 **PyTorch 自动微分** 计算 |
| **损失** | `loss_pde = MSE(residual, 0)` | 把物理约束变成**可直接最小化的标量** |

> PINN 精髓：  
> **“让物理方程成为网络的一层损失”**，而不是后验校验。

---

### 5.3 参数反演 (Parameter Inference)

| 可学习对象 | 代码 | 结果 |
|---|---|---|
| **未知物理常数 ν** | `self.nu = nn.Parameter(...)` | 与网络权重 **同步接受梯度** |
| **梯度来源** | `total_loss.backward()` | PyTorch 自动计算 ∂loss/∂ν |
| **最终效果** | `optimizer.step()` | **反推出最合理的 ν**，实现**逆问题求解** |

> 🔍 **一句话总结**：  
> 网络不仅学“解”，还学“物理常数”，真正做到 **“数据 + 物理” 双向自洽**。

In [2]:
class MLP(nn.Module):
    def __init__(self, layer_dims):
        super().__init__()
        self.layyers = nn.ModuleList()
        for i in range(len(layer_dims) - 1):
            self.layyers.append(nn.Linear(layer_dims[i], layer_dims[i + 1]))
        self.activation = nn.Tanh()

    def forward(self, x):
        for layer in self.layyers[:-1]:
            x = self.activation(layer(x))
        x = self.layyers[-1](x)
        return x


MLP.forward 的职责很纯粹：就是执行一个标准的多层感知机的前向计算。

PINN_Burgers.forward 的职责是“编排”：它负责准备 MLP 需要的输入格式，并调用 MLP 来完成计算。

PINN_Burgers.compute_pde_residual 的职责是“物理约束”：它调用 forward 得到解 u，然后利用自动微分来计算和物理方程相关的损失。

## 1. __init__ —— 网络初始化


| 步骤          | 关键动作                             | 设计意图                           |
| ----------- | -------------------------------- | ------------------------------ |
| 1️⃣ 初始化父类   | `super().__init__()`             | 遵循 PyTorch 模块规范                |
| 2️⃣ 创建核心网络  | `self.network = MLP(layer_dims)` | **模块化**：复用已写好的 `MLP`，避免重复造轮子   |
| 3️⃣ 声明可学习参数 | `self.nu = nn.Parameter(...)`    | **逆问题核心**：让 **黏度系数 ν** 也参与反向传播 |

🔍 亮点：把物理常数 ν 设为 nn.Parameter，网络不仅拟合解 u(x,t)，还能从数据里反推最合适的物理参数。



## 2. forward —— 数据前向传播

| 步骤 | 关键动作 | 设计意图 |
|---|---|---|
| **1️⃣ 拼接输入** | `torch.cat([x, t], dim=1)` | **时空整合**：把空间坐标 `x` 和时间坐标 `t` 组合成 `[batch, 2]` 的单一输入，这是神经网络处理时空问题的常见套路。 |
| **2️⃣ 核心计算** | `self.network(inputs)` | **功能解耦**：将真正的数据映射逻辑完全交给内部的 `MLP`，主网络只做“包装器”，便于后续替换或升级。 |

> 🔍 **亮点**：整个 `forward` 只有两行代码，却清晰展示了 **如何通过极简的输入预处理，把一个通用 MLP 迅速适配到时空预测任务**。

---

## 3. compute_pde_residual —— 物理信息核心

| 步骤 | 关键动作 | 设计意图 |
|---|---|---|
| **1️⃣ 开启微分** | `x.requires_grad_(True)` | **启用自动微分**：告诉 PyTorch 需要为 `x`、`t` 计算梯度，为后续求偏导做准备。 |
| **2️⃣ 计算偏导数** | `torch.autograd.grad(...)` | **核心机制**：利用 PyTorch 的自动微分功能，轻松求出 `u_t`、`u_x`、`u_xx`，这是 PINN 的关键技术。 |
| **3️⃣ 计算残差** | `residual = u_t + u * u_x - self.nu * u_xx` | **物理约束**：将偏导数直接代入 Burgers 方程，得到方程的不平衡量；该残差作为损失项，强制网络遵守物理定律。 |

> 🔍 **亮点**：该方法是整个 PINN 的精髓——**把数学物理方程直接转成神经网络的可计算损失**，使模型在拟合数据的同时，也学会遵守物理规律。

## 4. 残差的物理意义与训练作用

| 概念 | 公式 / 代码 | 作用 |
|---|---|---|
| **Burgers 方程** | $u_t + u u_x - \nu u_{xx} = 0$ | 描述一维粘性流体运动的物理规律 |
| **残差（residual）** | `residual = u_t + u * u_x - self.nu * u_xx` | 网络预测代入方程后 **左侧 ≠ 0** 的剩余量 |
| **训练目标** | $\mathcal{L}_{\text{pde}} = \text{MSE}(\text{residual},\ 0)$ | 将残差作为额外损失项，**逼迫网络让残差 → 0** |
| **最终效果** | —— | 模型在**拟合观测数据**的同时，也**严格遵守物理方程** |

> 🔍 **一句话总结**：残差把“物理定律”翻译成可微的损失函数，让优化器在最小化数据误差的同时，也最小化“违反物理”的程度。

In [3]:
class PINN_Burgers(nn.Module):
    def __init__(self, layer_dims, true_nu=None):
        super().__init__()
        self.network = MLP(layer_dims)
        
        # 将 nu 定义为一个可学习的参数
        # 我们用一个猜测值进行初始化，例如 0.1
        # 如果提供了 true_nu，则使用它（用于调试/测试正向问题）
        initial_nu = 0.1 if true_nu is None else true_nu
        self.nu = nn.Parameter(torch.tensor([initial_nu], dtype=torch.float32, requires_grad=True))
    
    def forward(self, x, t):
        # 拼接 x 和 t 以创建网络输入
        inputs = torch.cat([x, t], dim=1)
        return self.network(inputs)
    
    def compute_pde_residual(self, x, t):
        # 为输入设置 requires_grad=True 以计算导数
        x.requires_grad_(True)
        t.requires_grad_(True)
        
        u = self.forward(x, t)
        
        # 使用自动微分计算导数
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        
        # 伯格斯方程残差
        residual = u_t + u * u_x - self.nu * u_xx
        return residual


In [7]:
# -- 2. 主训练脚本 --
if __name__ == '__main__':
    # 设备设置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
 # --- 数据加载和准备 ---
    data = np.load('burgers_shock_solution.npz')
    x_data = torch.tensor(data['x'], dtype=torch.float32) # 形状: [nx, 1]
    t_data = torch.tensor(data['t'], dtype=torch.float32) # 形状: [nt, 1]
    u_solution = torch.tensor(data['u'], dtype=torch.float32) # 形状: [nt, nx]

    # 创建用于训练的坐标网格
    # 我们将在所有数据点上进行训练
    T, X = torch.meshgrid(t_data.squeeze(), x_data.squeeze(), indexing='ij')
    
   # 准备训练数据三元组 (x, t, u)
    x_train = X.reshape(-1, 1)
    t_train = T.reshape(-1, 1)
    u_train = u_solution.reshape(-1, 1)

# 将所有训练数据移动到选定的设备
    x_train = x_train.to(device)
    t_train = t_train.to(device)
    u_train = u_train.to(device)


  # --- 使用 TensorDataset 和 DataLoader 创建小批量数据 ---
    # 定义批次大小，您可以根据您的GPU显存进行调整
    # batch_size = 2048
    # train_dataset = TensorDataset(x_train, t_train, u_train)
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# --- 模型、损失和优化器设置 ---
# 对于逆向问题，我们不提供真实的 nu
# pinn_model.parameters() 是一个“参数收集器,返回一个包含所有找到的参数的集合
pinn_model = PINN_Burgers(layer_dims=[2, 20, 1]).to(device)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

Using device: cuda


In [8]:
 # --- 训练循环 ---
epochs = 20000
# 定义一个裁剪阈值
clip_value = 1.0 
for epoch in range(epochs):
    optimizer.zero_grad()

    # 1. 数据损失
    u_pred = pinn_model(x_train, t_train)
    data_loss = loss_fn(u_pred, u_train)
# 2. 物理损失
    # 对于物理损失，我们可以使用相同的点或采样新的点。
    # 这里为了简单起见，我们使用相同的训练点。
    pde_residual = pinn_model.compute_pde_residual(x_train, t_train)
    physics_loss = loss_fn(pde_residual, torch.zeros_like(pde_residual))

    # 或者 0.01，这是一个需要调整的超参数
    lambda_physics = 10
    physics_loss = lambda_physics * physics_loss
    # 总损失（您可以添加一个权重，例如 total_loss = data_loss + 0.1 * physics_loss）
    total_loss = data_loss + physics_loss

    total_loss.backward()

    # 在更新之前进行梯度裁剪
    torch.nn.utils.clip_grad_norm_(pinn_model.parameters(), clip_value)
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        # 秘密答案是 0.07。让我们看看我们的猜测有多接近！
        print(f"Epoch [{epoch+1}/{epochs}], Total Loss: {total_loss.item():.4f}, "
                f"Data Loss: {data_loss.item():.4f}, Physics Loss: {physics_loss.item():.4f}, "
                f"Predicted nu: {pinn_model.nu.item():.4f}")
print("\nTraining finished!")
print(f"The final predicted viscosity nu is: {pinn_model.nu.item():.5f}")
print(f"(The true value was 0.07)") 

    


C:\Users\27346\AppData\Roaming\Python\Python311\site-packages\torch\autograd\graph.py:824: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:181.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1000/20000], Total Loss: 0.1132, Data Loss: 0.1025, Physics Loss: 0.0107, Predicted nu: 0.3449
Epoch [2000/20000], Total Loss: 0.0647, Data Loss: 0.0564, Physics Loss: 0.0083, Predicted nu: 0.3075
Epoch [3000/20000], Total Loss: 0.0468, Data Loss: 0.0396, Physics Loss: 0.0072, Predicted nu: 0.2517
Epoch [4000/20000], Total Loss: 0.0372, Data Loss: 0.0314, Physics Loss: 0.0058, Predicted nu: 0.2128
Epoch [5000/20000], Total Loss: 0.0322, Data Loss: 0.0267, Physics Loss: 0.0054, Predicted nu: 0.1936
Epoch [6000/20000], Total Loss: 0.0297, Data Loss: 0.0241, Physics Loss: 0.0055, Predicted nu: 0.1764
Epoch [7000/20000], Total Loss: 0.0281, Data Loss: 0.0225, Physics Loss: 0.0056, Predicted nu: 0.1625
Epoch [8000/20000], Total Loss: 0.0269, Data Loss: 0.0213, Physics Loss: 0.0056, Predicted nu: 0.1506
Epoch [9000/20000], Total Loss: 0.0260, Data Loss: 0.0203, Physics Loss: 0.0057, Predicted nu: 0.1393
Epoch [10000/20000], Total Loss: 0.0238, Data Loss: 0.0189, Physics Loss: 0.0050, 

In [10]:
# 保存标准 PINN 模型
PATH = r"D:\A-Code\GitHubCode\Pytorch-PINN-Learning\model\stage3_ipinn_model.pth"
torch.save(pinn_model.state_dict(), PATH)